In [ ]:
cur = engine.execute(
    select([users.c.name, addr.c.email]).select_from(users.join(addr)))
cur.fetchall()

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base

In [ ]:
engine = create_engine('sqlite:///ex4db', echo=True)

In [ ]:
base = declarative_base()

In [ ]:
# Metadata Clas에 등록된(생성한) table  객체 초기화 할때
base.metadata.clear()

In [ ]:
class User(base):
    __tablename__ = 'User'
    
    uid = Column('uid', Integer, primary_key = True)
    name = Column('name', String, nullable=True)
    password = Column('pass', String, nullable=True)
    
    def __repr__(self):
        return 'UID={0}, Name={1}, PASS={2}'.format(
            self.uid, self.name, self.password)
    

In [ ]:
user1 = User(name='test', password='testtest')

In [ ]:
user1

In [ ]:
from sqlalchemy.orm import sessionmaker

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
# base.metadata.create_all(bind=engine)

In [ ]:
session.add(user1)

In [ ]:
session.commit()

In [ ]:
user1.name = '개똥이이'

In [ ]:
session.dirty

In [ ]:
session.is_modified(user1)

In [ ]:
session.commit()

In [ ]:
session.add_all([User(name='test2', password='asdfasdf'),
               User(name='test3', password='asdffdsad')])

In [ ]:
cur = session.query(User)
[_ for _ in cur]

In [ ]:
[_ for _ in session.query(User).filter(User.name.like('test%'))]

In [ ]:
# class Artist(base):
#     __tablename__ = 'ARTIST'
    
#     pk = Column('ID', Integer, primary_key=True)
#     name = Column('Name', String)
    
#     def __repr__(self):
#         return 'PK:{0}, Name:{1}'.format(self.pk, self.name)
    
# class Genre(base):
#     __tablename__ = 'GENRE'
    
#     pk = Column('ID', Integer, primary_key=True)
#     name = Column('Name', String)
    
#     def __repr__(self):
#         return 'PK:{0}, Name:{1}'.format(self.pk, self.name)
    
# class Album(base):
#     __tablename__ = 'ALBUM'
    
#     pk = Column('ID', Integer, primary_key=True)
#     name = Column('Name', String)
#     fk = Column('FK', Integer, ForeignKey('ARTIST.ID'))
    
#     def __repr__(self):
#         return 'PK:{0}, Name:{1}, FK:{2}'.format(self.pk, self.name, self.fk)
    
# class Track(base):
#     __tablename__ = 'TRACK'
    
#     pk = Column('ID', Integer, primary_key=True)
#     name = Column('Name', String)
#     fk1 = Column('FK', Integer, ForeignKey('ALBUM.ID'))
#     fk2 = Column('FK', Integer, ForeignKey('GENRE.ID'))
    
#     def __repr__(self):
#         return 'PK:{0}, Name:{1}, FK1:{2}, FK:{3}'.format(self.pk, self.name, self.fk1, self.fk2)
    


In [ ]:
base.metadata.clear()

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///ex5db', echo=True)
base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
class Artist(base):
    __tablename__ = "Artist"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
class Album(base):
    __tablename__ = "Album"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey('Artist.id'))
    
class Genre(base):
    __tablename__ = "Genre"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
class Track(base):
    __tablename__ = "Track"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    length = Column(String)
    rating = Column(Integer)
    count = Column(Integer)
    album_id = Column(Integer, ForeignKey('Album.id'))
    genre_id = Column(Integer, ForeignKey('Genre.id'))

In [ ]:
artist1 = Artist(name="Led Zeppllin")
artist2 = Artist(name="AC/DC")

session.add_all([artist1, artist2])
session.commit()

In [ ]:
album = [Album(title="IV", artist_id=artist1.id),
        Album(title="Who Made Who", artist_id=artist2.id)]

In [ ]:
session.add_all(album)

In [ ]:
session.commit()

In [ ]:
base.metadata.tables

In [ ]:
base.metadata.clear()

In [ ]:
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base

In [ ]:
engine = create_engine('sqlite:///ex5db', echo=True)
base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
class Artist(base):
    __tablename__ = 'Artist'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    albumlist = relationship('Album', back_populates='artist')
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}'.format(self.pk, self.name)
    
class Genre(base):
    __tablename__ = 'Genre'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    tracklist = relationship('Track', back_populates='genre')
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}'.format(self.pk, self.name)
    
class Album(base):
    __tablename__ = 'Album'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    fk = Column('FK', Integer, ForeignKey('Artist.ID'))
    artist = relationship('Artist', back_populates='albumlist',
                          uselist=False)
    tracklist = relationship('Track', back_populates='album')
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}, FK:{2}'.format(self.pk, self.name, self.fk)
    
class Track(base):
    __tablename__ = 'Track'
    
    pk = Column('ID', Integer, primary_key=True)
    name = Column('NAME', String)
    fk1 = Column('FK1', Integer, ForeignKey('Album.ID'))
    fk2 = Column('FK2', Integer, ForeignKey('Genre.ID'))
    album = relationship('Album', back_populates='tracklist', uselist = False)
    genre = relationship('Genre', back_populates='tracklist', uselist = False)
    
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}, FK1:{2}, FK2:{3}'.format(self.pk, self.name, self.fk1, self.fk2)
    


In [1]:
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base

In [2]:
engine = create_engine('sqlite:///playlist.db', echo=True)
base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
class Artist(base):
    __tablename__ = 'Artist'
    
    pk = Column('id', Integer, primary_key=True)
    name = Column('title', String)
    albumlist = relationship('Album', back_populates='artist')
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}'.format(self.pk, self.name)
    
class Genre(base):
    __tablename__ = 'Genre'
    
    pk = Column('id', Integer, primary_key=True)
    name = Column('title', String)
    tracklist = relationship('Track', back_populates='genre')
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}'.format(self.pk, self.name)
    
class Album(base):
    __tablename__ = 'Album'
    
    pk = Column('id', Integer, primary_key=True)
    name = Column('title', String)
    fk = Column('Artist_id', Integer, ForeignKey('Artist.id'))
    artist = relationship('Artist', back_populates='albumlist',
                          uselist=False)
    tracklist = relationship('Track', back_populates='album')
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}, FK:{2}'.format(self.pk, self.name, self.fk)
    
class Track(base):
    __tablename__ = 'Track'
    
    pk = Column('id', Integer, primary_key=True)
    name = Column('title', String)
    fk1 = Column('Album_id', Integer, ForeignKey('Album.id'))
    fk2 = Column('Genre_id', Integer, ForeignKey('Genre.id'))
    album = relationship('Album', back_populates='tracklist', uselist = False)
    genre = relationship('Genre', back_populates='tracklist', uselist = False)
    
    
    def __repr__(self):
        return 'PK:{0}, NAME:{1}, FK1:{2}, FK2:{3}'.format(self.pk, self.name, self.fk1, self.fk2)
    


In [4]:
genrelist = session.query(Genre).all()

2020-07-15 12:05:24,909 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-15 12:05:24,912 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:05:24,914 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-15 12:05:24,915 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 12:05:24,916 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 12:05:24,917 INFO sqlalchemy.engine.base.Engine SELECT "Genre".id AS "Genre_id", "Genre".title AS "Genre_title" 
FROM "Genre"
2020-07-15 12:05:24,918 INFO sqlalchemy.engine.base.Engine ()


In [7]:
# session.add(Track(name='TEST'),
#            fk1 = genrelist[1].tracklist[0].album.pk,
#            fk2=genrelist[1].pk)